In [1]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics

# Divide train into 90 % training and 10 % testing

In [2]:
all_features = ["text_ tokens", "tweet_id" ,"language", "tweet_timestamp", "engaged_with_user_id",\
               "engaging_user_id", "engagee_follows_engager", "retweet_with_comment_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [0, 2, 7, 8, 9, 14, 19, 20], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))
val_set['retweet_with_comment_bool'] = val_set.retweet_with_comment_timestamp.fillna(0)
val_set[val_set.retweet_with_comment_bool != 0.0] = 1.0
gt = list(val_set.retweet_with_comment_bool)
pr = [0.5] * len(gt)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [ ]:
print(prauc)
print(rce)